In [1]:
# need to install for colab

!pip install swifter
!pip install pyDRMetrics
!pip install keras-tuner
!pip install rasterio
!pip install dask[dataframe]
!pip install geopandas
!pip install h3
!pip install tqdm
!pip install rioxarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached rasterio-1.2.10-cp37-cp37m-manylinux1_x86_64.whl (19.3 MB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached affine-2.3.1-py2.py3-none-any.whl (16 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 32.2 MB/s 
     |████████████████████████████████| 6.3 M

In [1]:
!nvidia-smi

Tue Aug  9 22:31:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/dssg/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install repo

In [2]:
cd stc_unicef_cpi

/content/drive/My Drive/dssg/stc_unicef_cpi


In [3]:
!pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/My Drive/dssg/stc_unicef_cpi
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for stc-unicef-cpi: filename=stc_unicef_cpi-0.0.1-py3-none-any.whl size=60873 sha256=c56eaf93694bdc101271ff5ac2b72dd7cc0566ded755823e8ef4a6b1befe9950
  Stored in directory: /tmp/pip-ephem-wheel-cache-ule8m1f3/wheels/bf/94/9c/40759959ec8f02629aa5962548767ba6e8d7c3aa

# Imports

In [4]:
import tensorflow
from tensorflow import keras
from keras.layers import Conv2D, UpSampling2D, MaxPooling2D
import numpy as np
from tensorflow.keras.optimizers import Adam
import gc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt
from pyDRMetrics.pyDRMetrics import *
from stc_unicef_cpi.data import cv_loaders as cvl

from stc_unicef_cpi.features import autoencoder_features as af
import pandas as pd

random_state=0
np.random.seed(random_state)

In [5]:
os.chdir('/content/drive/My Drive/dssg/')

In [6]:
# garbage collection
# done in between cells to clear ram

gc.collect()

102

# Get Hex codes

In [7]:
df = pd.read_csv('Data/nga_clean_expanded.csv')
hex_codes = list(df.hex_incl_nbrs.values)
len(hex_codes)

17379

In [8]:
del df; gc.collect()

50

# Get training set for autoencoder

In [10]:
train_img_arr = af.get_train_data("nga_tiffs", hex_codes)
train_img_arr.shape

Dataset progress::   0%|          | 0/19 [00:00<?, ?it/s]

Location progress::   0%|          | 0/17379 [00:00<?, ?it/s]

Location progress::   0%|          | 0/17379 [00:00<?, ?it/s]

Location progress::   0%|          | 0/17379 [00:00<?, ?it/s]

Location progress::   0%|          | 0/17379 [00:00<?, ?it/s]

...

Location progress::   0%|          | 0/17379 [00:00<?, ?it/s]

Location progress::   0%|          | 0/17379 [00:00<?, ?it/s]

Location progress::   0%|          | 0/17379 [00:00<?, ?it/s]

Location progress::   0%|          | 0/17379 [00:00<?, ?it/s]

(17379, 16, 16, 62)

In [11]:
# checking for anomalies that could result in nan/inf autoencoder loss

np.min(train_img_arr[~np.isnan(train_img_arr)]), np.max(train_img_arr[~np.isnan(train_img_arr)])

(-20.3042722857505, 129.98641000732368)

In [15]:
gc.collect()

416

# Tune Model HPs (only if necessary)

In [13]:
# tune hyperparams for model if needed; else use default values in get_trained_autoencoder
# runs for ~ 8 mins (with GPU) for dataset with 17k samples
# see func definition for customizing learning_rate, epochs & batch_size options

best_hps = af.get_best_hyperparameters(train_img_arr)

Trial 7 Complete [00h 00m 08s]
val_loss: 6.632408142089844

Best val_loss So Far: 6.407967567443848
Total elapsed time: 00h 01m 05s
INFO:tensorflow:Oracle triggered exit

  Best LR: 0.001
  Best Batch Size: 128.
  
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 16, 16, 32)        17888     
                                                                 
 mp1 (MaxPooling2D)          (None, 8, 8, 32)          0         
                                                                 
 conv2 (Conv2D)              (None, 8, 8, 16)          4624      
                                                                 
 mp2 (MaxPooling2D)          (None, 4, 4, 16)          0         
                                                                 
 conv3 (Conv2D)              (None, 4, 4, 8)           1160      
                                     

In [22]:
gc.collect()

268

# Get trained model

In [ ]:
save_dir = "models"
model_name = "autoencoder_nga"

# get model with either tuned or default HPs
 
if best_hps:
  af.get_trained_autoencoder(input_data=train_img_arr,
                             batch_size=best_hps['batch size'],
                             epochs=best_hps['epochs'],
                             learning_rate=best_hps['learning rate'],
                             save_dir=save_dir,
                             model_name=model_name
                             )
else:
    af.get_trained_autoencoder(input_data=train_img_arr,
                             save_dir=save_dir,
                             model_name=model_name
                             )

In [26]:
gc.collect()

506

# Generate features using trained model

In [11]:
# takes about 3 mins with GPU, for 17k samples
# processed in batches of 4096 (default value); could try larger values for quicker processing?
# gives CPLE_NotSupported in driver GTiff in Colab
# error is specific to IPython interfaces; code still works correctly

trained_autoencoder_dir = "models"
model_name = "autoencoder_nga"
tiff_files_dir = "nga_tiffs"


features = af.get_encoded_features(trained_autoencoder_dir=trained_autoencoder_dir,
                                   model_name=model_name, hex_codes=np.array(hex_codes),
                                   tiff_files_dir=tiff_files_dir)

Dataset progress::   0%|          | 0/19 [00:00<?, ?it/s]

Location progress::   0%|          | 0/4096 [00:00<?, ?it/s]

Location progress::   0%|          | 0/4096 [00:00<?, ?it/s]

Location progress::   0%|          | 0/4096 [00:00<?, ?it/s]

Location progress::   0%|          | 0/4096 [00:00<?, ?it/s]

...

Location progress::   0%|          | 0/4096 [00:00<?, ?it/s]

Location progress::   0%|          | 0/4096 [00:00<?, ?it/s]

Location progress::   0%|          | 0/4096 [00:00<?, ?it/s]

Location progress::   0%|          | 0/4096 [00:00<?, ?it/s]

Location progress::   0%|          | 0/4096 [00:00<?, ?it/s]

In [17]:
gc.collect()

701

In [13]:
features.shape

(17379, 32)

In [15]:
# saving as dataframe for now, whose columns are encoded features
# and rows are samples, i.e. hexagons
# index of csv can be used to join autoencoder features with other features

res = pd.DataFrame(
    data = features,
    columns = ['F_'+str(i) for i in range(features.shape[1])],
    index = hex_codes
)
res.head(2)

,F_0,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,F_9,...,F_22,F_23,F_24,F_25,F_26,F_27,F_28,F_29,F_30,F_31
609534208263585791,3.896168,11.073842,13.581891,12.117510,4.456576,7.543083,0.0,10.051905,25.152727,9.210425,...,0.0,0.0,28.343771,5.141608,0.0,12.085501,7.922971,21.971462,3.042949,0.0
609534208330694655,9.032895,6.847852,2.797609,4.082599,0.000000,3.730211,0.0,11.784893,14.154691,12.027953,...,0.0,0.0,26.073893,1.315042,0.0,13.483556,4.298766,8.221313,10.386462,0.0


In [16]:
res.to_csv('Data/encodings/alltiffs_nga_train_encodings.csv')